### CephFS 

Wir verwenden [Host Storage Cluster](https://rook.io/docs/rook/v1.11/CRDs/Cluster/host-cluster/)

Überprüfen, ob die CephFS-CSI-Plugins korrekt laufen:

In [ ]:
%%bash
kubectl -n rook-ceph get pods -l app=csi-cephfsplugin

Zuerst brauchen wie ein Filesystem

In [3]:
%%bash
kubectl apply -f https://raw.githubusercontent.com/rook/rook/refs/heads/release-1.11/deploy/examples/filesystem-test.yaml

cephfilesystem.ceph.rook.io/myfs created


Prüfen, ob CephFS korrekt eingerichtet ist
Überprüfe, ob CephFS bereits läuft:

In [4]:
%%bash
kubectl -n rook-ceph get CephCluster,cephfilesystems,CephFilesystemSubVolumeGroup
kubectl -n rook-ceph describe CephCluster

NAME                                  DATADIRHOSTPATH   MONCOUNT   AGE     PHASE         MESSAGE                 HEALTH   EXTERNAL   FSID
cephcluster.ceph.rook.io/my-cluster   /var/lib/rook     1          7m29s   Progressing   Configuring Ceph Mons                       

NAME                               ACTIVEMDS   AGE   PHASE
cephfilesystem.ceph.rook.io/myfs   1           4s    Progressing
Name:         my-cluster
Namespace:    rook-ceph
Labels:       <none>
Annotations:  <none>
API Version:  ceph.rook.io/v1
Kind:         CephCluster
Metadata:
  Creation Timestamp:  2025-02-08T18:50:37Z
  Finalizers:
    cephcluster.ceph.rook.io
  Generation:        2
  Resource Version:  2560
  UID:               1771db5a-70aa-44dc-ab08-88dcd025deee
Spec:
  Ceph Version:
    Allow Unsupported:  true
    Image:              quay.io/ceph/ceph:v17
  Cleanup Policy:
    Sanitize Disks:
  Crash Collector:
    Disable:  true
  Dashboard:
    Enabled:           true
  Data Dir Host Path:  /var/lib/rook
 

*Evtl. könnten die Zugriffsberechtigungen RBAC nicht vorhanden sein, deshalb setzen wir diese*

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: ceph-csi-provisioner
rules:
  - apiGroups: [""]
    resources: ["persistentvolumes"]
    verbs: ["get", "list", "watch", "create", "delete", "patch", "update"]
  - apiGroups: ["storage.k8s.io"]
    resources: ["storageclasses"]
    verbs: ["get", "list", "watch"]
  - apiGroups: ["storage.k8s.io"]
    resources: ["volumeattachments"]
    verbs: ["get", "list", "watch", "patch", "update"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: ceph-csi-provisioner-binding
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: ceph-csi-provisioner
subjects:
  - kind: ServiceAccount
    name: rook-csi-cephfs-provisioner
    namespace: rook-ceph
EOF
# braucht Restart
kubectl -n rook-ceph delete pod -l app=csi-cephfsplugin-provisioner

### StorageClass für Rook-Ceph erstellen.

Mit dieser werden keine PV erstellt.

Alternative microk8s-hostpath vom Addon hostpath-storage verwenden

In [2]:
%%bash
kubectl apply -f https://raw.githubusercontent.com/rook/rook/refs/heads/release-1.11/deploy/examples/csi/rbd/storageclass-test.yaml

cephblockpool.ceph.rook.io/replicapool created
storageclass.storage.k8s.io/rook-ceph-block created


In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: rook-cephfs 
provisioner: rook-ceph.cephfs.csi.ceph.com
parameters:
  clusterID: rook-ceph
  fsName: myfs
  pool: myfs-data0
  csi.storage.k8s.io/provisioner-secret-name: rook-csi-cephfs-provisioner
  csi.storage.k8s.io/provisioner-secret-namespace: rook-ceph
  csi.storage.k8s.io/node-stage-secret-name: rook-csi-cephfs-node
  csi.storage.k8s.io/node-stage-secret-namespace: rook-ceph
  csi.storage.k8s.io/fstype: ext4
reclaimPolicy: Delete
EOF

### PersistentVolume und Persistent Volume Claim (PVC) für Ceph RBD erstellen

**Hinweis** PersistentVolume sollte eigentlich automatisch erstellt werden.

### Hole die benötigten Informationen für das PV

Die Ceph-Monitor-Adressen

In [ ]:
%%bash
kubectl -n rook-ceph get cm rook-ceph-mon-endpoints -o jsonpath='{.data.data}'

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: PersistentVolume
metadata:
  name: cephfs-pv
spec:
  capacity:
    storage: 5Gi  # Größe des Volumes anpassen
  accessModes:
    - ReadWriteMany  # CephFS unterstützt RWX
  persistentVolumeReclaimPolicy: Retain
  storageClassName: rook-cephfs   # Muss mit deiner StorageClass übereinstimmen
  csi:
    driver: rook-ceph.cephfs.csi.ceph.com
    volumeHandle: myfs  # Name des CephFS-Dateisystems
    volumeAttributes:
      clusterID: rook-ceph
      fsName: myfs
      pool: myfs-data0
      provisionVolume: "true" 
      monitors: "10.152.183.133:6789"
    nodeStageSecretRef:
      name: rook-csi-cephfs-node
      namespace: rook-ceph
EOF


In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: ceph-pvc

spec:
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 5Gi
  storageClassName: rook-cephfs
  # volumeName: cephfs-pv  # Name des vorher erstellten PVs
EOF

Wir kontrollieren ob alles sauber erstellt wurde

In [ ]:
%%bash
kubectl describe pvc ceph-pvc

Pod mit Ceph-Speicher starten

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: ceph-test-pod

spec:
  containers:
  - name: busybox
    image: busybox
    command: ["/bin/sh", "-c", "while true; do echo 'Ceph läuft!' >> /mnt/test/data.txt; sleep 10; done"]
    volumeMounts:
    - mountPath: "/mnt/test"
      name: ceph-storage
  volumes:
  - name: ceph-storage
    persistentVolumeClaim:
      claimName: ceph-pvc
EOF

In [ ]:
%%bash
kubectl describe pod ceph-test-pod

---

### Aufräumen

In [1]:
%%bash
kubectl delete pod ceph-test-pod 
kubectl delete pvc ceph-pvc
kubectl delete pv cephfs-pv 
kubectl delete sc rook-cephfs


Error from server (NotFound): pods "ceph-test-pod" not found
Error from server (NotFound): persistentvolumeclaims "ceph-pvc" not found
Error from server (NotFound): persistentvolumes "cephfs-pv" not found
Error from server (NotFound): storageclasses.storage.k8s.io "rook-cephfs" not found


CalledProcessError: Command 'b'kubectl delete pod ceph-test-pod \nkubectl delete pvc ceph-pvc\nkubectl delete pv cephfs-pv \nkubectl delete sc rook-cephfs\n'' returned non-zero exit status 1.

### Tests

In [ ]:
%%bash
kubectl -n rook-ceph exec deploy/rook-ceph-tools -- ceph status
